Isoflow

In [ ]:
import sys

import scanpy as sc
import anndata as ad

In [ ]:
# import the dataset
#
DATA_PATH = "/dtu/blackhole/06/213542/paperdata/pbmc3k_train.h5ad"
adata = sc.read_h5ad(DATA_PATH)

#remove genes with low count
sc.pp.filter_genes(adata, min_cells=20)